<a href="https://colab.research.google.com/github/richardGonzalez-std/InventarioKivyMVC/blob/main/tools/Build_APK_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIAM - Compilar APK en Google Colab

**Primera vez:** ~30 min (descarga SDK/NDK)

**Siguientes veces:** ~5-10 min (usa cache de Drive)

---

## 1. Configuracion

In [ ]:
# ============================================
# EDITA ESTAS LINEAS
# ============================================
GITHUB_REPO = "https://github.com/richardGonzalez-std/InventarioKivyMVC.git"
BRANCH = "main"

# Usar cache de buildozer en Drive (RECOMENDADO)
USE_CACHE = True

print(f"Repo: {GITHUB_REPO}")
print(f"Rama: {BRANCH}")
print(f"Cache: {'Activado' if USE_CACHE else 'Desactivado'}")

## 2. Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive montado')

## 3. Clonar repositorio

In [ ]:
import os
import shutil
from pathlib import Path

WORK_DIR = '/content/InventarioKivyMVC'
CACHE_DIR = '/content/drive/MyDrive/buildozer_cache'

# Limpiar directorio de trabajo
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

# Clonar repo
!git clone --branch {BRANCH} --depth 1 {GITHUB_REPO} {WORK_DIR}

os.chdir(WORK_DIR)
print(f'\nProyecto clonado en: {WORK_DIR}')

## 4. Restaurar cache de buildozer (si existe)

In [ ]:
if USE_CACHE:
    buildozer_local = '/root/.buildozer'

    if os.path.exists(CACHE_DIR):
        print('Restaurando cache de buildozer desde Drive...')
        print('(Esto puede tomar unos minutos)')
        
        if os.path.exists(buildozer_local):
            shutil.rmtree(buildozer_local)
        
        shutil.copytree(CACHE_DIR, buildozer_local)
        
        # Calcular tamaño
        size_mb = sum(f.stat().st_size for f in Path(buildozer_local).rglob('*') if f.is_file()) / 1024 / 1024
        print(f'Cache restaurado: {size_mb:.0f} MB')
    else:
        print('No hay cache previo en Drive.')
        print('Primera compilacion sera lenta (~30 min)')
        print('El cache se guardara automaticamente al terminar.')
else:
    print('Cache desactivado')

## 5. Instalar dependencias

In [ ]:
print('Instalando dependencias del sistema...')
!apt-get update -qq
!apt-get install -y -qq \
    python3-pip build-essential git python3-dev \
    ffmpeg libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev \
    libportmidi-dev libswscale-dev libavformat-dev libavcodec-dev zlib1g-dev \
    libgstreamer1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good \
    zip unzip openjdk-17-jdk autoconf libtool pkg-config \
    libncurses5-dev libncursesw5-dev cmake libffi-dev libssl-dev \
    gettext autopoint libltdl-dev libtool-bin lld m4

!pip install --upgrade pip -q
!pip install --upgrade buildozer cython==0.29.36 -q
print('Dependencias instaladas')

## 6. Compilar APK

In [ ]:
import time

os.chdir(WORK_DIR)

start = time.time()
!buildozer android debug 2>&1 | tee build.log
elapsed = time.time() - start

print(f'\nTiempo de compilacion: {elapsed/60:.1f} minutos')

## 7. Guardar cache en Drive (para proximas compilaciones)

In [ ]:
if USE_CACHE:
    buildozer_local = '/root/.buildozer'

    if os.path.exists(buildozer_local):
        print('Guardando cache de buildozer en Drive...')
        print('(Esto puede tomar unos minutos pero ahorrara tiempo en futuras compilaciones)')

        if os.path.exists(CACHE_DIR):
            print('Eliminando cache anterior...')
            shutil.rmtree(CACHE_DIR)

        shutil.copytree(buildozer_local, CACHE_DIR)

        size_mb = sum(f.stat().st_size for f in Path(CACHE_DIR).rglob('*') if f.is_file()) / 1024 / 1024
        print(f'\nCache guardado: {size_mb:.0f} MB')
        print(f'Ubicacion: {CACHE_DIR}')
    else:
        print('No se encontro directorio .buildozer para guardar')

## 8. Guardar APK en Drive

In [ ]:
bin_path = Path(WORK_DIR) / 'bin'
apks = list(bin_path.glob('*.apk')) if bin_path.exists() else []

if apks:
    apk_path = apks[0]
    size_mb = apk_path.stat().st_size / 1024 / 1024
    print(f'APK: {apk_path.name}')
    print(f'Tamano: {size_mb:.1f} MB')

    output_dir = '/content/drive/MyDrive/APKs'
    os.makedirs(output_dir, exist_ok=True)
    dest = f'{output_dir}/{apk_path.name}'
    shutil.copy2(apk_path, dest)
    print(f'\nGuardado en: {dest}')
else:
    print('ERROR: No se genero el APK')
    print('\nErrores encontrados:')
    !grep -i "error" build.log | tail -20

## 9. Descargar APK

In [ ]:
if apks:
    from google.colab import files
    files.download(str(apk_path))